# Vegetation Status Mapping in Uganda's Urban Metropolitans.
Mapping vegetation status using remote sensing vegetation indices has significantly contributed to the provision of actional measures for improving environmental monitoring and natural resource management. This script takes the initiative to exploit remote sensing technologies by implementing industry vegetation indices to derive vegetation status coverage. (The script allows analysis for other study areas by assigning the district name as a variable). With a focus on Uganda urban metropolitans, this analysis focuses on Gulu district which is one of districts that recnt achieved city status.

This expiremental notebook uses the [Sentinel-2 L2A surface reflectance](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR) data products available in the GEE asset catalog from **2017-03-28** to **present** to characterise indicators of vegetation status. This study is based on **2023-01-01** to **2024-01-01**

In [26]:
import os
import subprocess
import calendar
from datetime import datetime
# This module is for displaying results interactively:
try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Earth Engine module
try:
    import ee
except ImportError:
    print('Installing ee ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'earthengine-api'])
    
# This module extends Google Earth Engine functionality
try:
    import eemont
except ImportError:
    print('Installing eemont ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'eemont'])

In [34]:
ee.Authenticate()
ee.Initialize()
#Please follow intructions to get the authorisation code from Google earth engine

In [28]:
#data and study area
start_date = '2023-01-01'
end_date = '2024-01-01'
cloud_percentage = 10
feature_asset = 'projects/ee-edwig/assets/Uganda'
demarcation_type = 'DName2016'
region_name = 'GULU'
roi = ee.FeatureCollection(feature_asset) \
            .filter(ee.Filter.eq(demarcation_type, region_name))


In [29]:
def level_pick(options, statement):
        print(statement)
        for idx, element in enumerate(options):
            print("{}) {}".format(idx + 1, element))
        i = input("Enter a number: ")
        try:
            if 0 < int(i) <= len(options):
                return int(i) - 1
        except:
            pass
        return None

### Classification threshold parameters

In [30]:
thresholds_vi = ee.Image([0.2, 0.3, 0.4, 0.6, 0.8])
thresholds_lai = ee.Image([0.5, 1, 2, 3, 3.5])
thresholds_nmdi_veg = ee.Image([0.2, 0.4, 0.6, 0.8])
thresholds_nmdi_soil = ee.Image([0.7, 0.5, 0.3, 0.15])
thresholds_fpi = ee.Image([0.2, 0.4, 0.6, 0.8])
thresholds_ccci = ee.Image([0.1, 0.3, 0.6, 0.7, 0.9])

#### Visualization palettes and parameters

In [31]:
ndvi_palette = ['#D3D3D3','#9c8448','#cccc66','#9cab68','#306466']

lai_palette = ['DF923D', 'F1B555', '99B718', '004C00', '011301']

#nmdi_palette = ['#DED799', '#A6B7B2', '#6F97CC', '#0057FF']
nmdi_palette = ['#ADD8E6', '#0000FF', '#00008B', '#000080']

fpi_palette = ['#FF0000', '#FFD700', '#90EE90', '#008000', '#0000FF']

ccci_palette = ['#440154', '#3E518A', '#21908C', '#5DC863', '#FDE724']
# Visualization parameters

rgb_viz = {
            'bands': ['B4','B3','B2'],
            'min': 0,
            'max': 0.3,
            'gamma': 2
            }

ndvi_viz = {
            'max': 1, 
            'min': 0, 
            'palette': [
                       '#0000ff', '#DF923D', '#F1B555',
                       '#FCD163', '#99B718', '#74A901', 
                       '#66A000', '#529400', '#3E8601', 
                       '#207401','#056201', '#004C00', 
                       '#023B01', '#012E01', '#011D01', 
                       '#011301']
            }

fpi_viz = {
            'min': 0,
            'max': 1,
            'palette': fpi_palette,
            'opacity': 0.75
            }

ndvi_class_viz = {
            'min': 0, 
            'max': 4, 
            'palette': ndvi_palette,
            'opacity': 0.75
            }

lai_class_viz = {
            'min': 0, 
            'max': 4, 
            'palette': lai_palette,
            'opacity': 0.75
            }

nmdi_class_viz = {
            'min':0, 
            'max':4, 
            'palette':nmdi_palette,
            'opacity': 0.75
            }

fpi_class_viz = {
            'min': 0, 
            'max': 4, 
            'palette': fpi_palette,
            'opacity': 0.75
            }

ccci_class_viz = {
    'min': 0,
    'max': 4,
    'palette': ccci_palette,
}


# Legend dictionaries
legend_dict = {
    "Non Vegetated Areas/sparse": "#D3D3D3",
    "Low Vegetation Vigour": "#9c8448",
    "Moderate Vegetation Vigour": "#cccc66",
    "Moderate-high Vegetation Vigour": "#9cab68",
    "High Vegetation Vigour": "#306466",
}

lai_legend_dict = {
    "up to 50 m^2 per 100 m^2": "#DF923D",
    "up to 100 m^2 per 100 m^2": "#F1B555",
    "up to 200 m^2 per 100 m^2": "#99B718",
    "up to 300 m^2 per 100 m^2": "#004C00",
    ">= 350 m^2 per 100 m^2": "#011301",
}

nmdi_legend_dict = {  
    "Very dry canopy": "#ADD8E6",
    "Dry canopy": "#0000FF",
    "Wet canopy": "#00008B",
    "Very wet canopy": "#000080",
}


fpi_legend_dict = {
    "Very high": "#FF0000",
    "High": "#FFD700",
    "Medium": "#90EE90",
    "Low": "#008000",
    "Very low": "#0000FF",
}

ccci_legend_dict = {
    'Low Chlorophyll': ccci_palette[0],
    'Moderate Low Chlorophyll': ccci_palette[1],
    'Moderate Chlorophyll': ccci_palette[2],
    'Moderate High Chlorophyll': ccci_palette[3],
    'High Chlorophyll': ccci_palette[4],
}

## Vegetation Health

Evaluating the health of the vegetation canopy frequently requires employing various remote sensing indices, which prove useful in identifying problematic areas. For example, regions in a field with an exceptionally low NDVI rate may indicate issues with pests or plant diseases, while areas displaying an unusually high NDVI could signify the presence of weeds. The choice of the appropriate index depends on factors such as the type of crop and the specific application for which it will be used.

Sentinel-2 surface reflectance bands offer the capability to compute the following indices:

1. **Normalized Difference Vegetation Index (NDVI):** NDVI is a widely-used metric for quantifying vegetation greenness and vigor. Elevated NDVI values typically indicate robust and dense vegetation. References: https://ntrs.nasa.gov/citations/19740022614

2. **Enhanced Vegetation Index (EVI):** Designed to mitigate influences from soil and atmosphere, EVI represents an enhancement over NDVI. It proves valuable for evaluating overall vegetation health and vigor. References: https://doi.org/10.1016/S0034-4257(96)00112-5

3. **Normalized Difference Vegetation Index (kNDVI):** kNDVI is a measure related to Leaf Area Index (LAI), providing insights into the amount of leaf material in a canopy. It contributes valuable information about vegetation structure and health. References: https://www.science.org/doi/10.1126/sciadv.abc7447

4. **Modified Soil-Adjusted Vegetation Index (MSAVI):** . This index provides accurate results for fewer vegetation areas where NDVI provides invalid data due to the low chlorophyll content. Therefore, is more suitable to monitoring of crop conditions at their earliest developmental stages where there are less vegetation and low growth. References:https://doi.org/10.1016/0034-4257(94)90134-1https://doi.org/10.1016/0034-4257(94)90134-1




In [32]:
options = ["Normalized Difference Vegetation Index (NDVI)", "Enhanced Vegetation Index (EVI)", "Unified Normalized Difference Vegetation Index (kNDVI)","Modified Soil-Adjusted Vegetation Index (MSAVI)"]
index = level_pick(options, "Which index would you like to utilize for evaluating the health of the vegetation canopy?")
# I use option 2, the EVI which represents an enhancement over NDVI. It proves valuable for evaluating overall vegetation health and vigor. 
if index == 0:
    vi_index = 'NDVI'
elif index == 1:
    vi_index = 'EVI'
elif index == 2:
    vi_index = 'kNDVI'
elif index == 3:
    vi_index = 'MSAVI'
else:
    print('Enter a valid input')

Which index would you like to utilize for evaluating the health of the vegetation canopy?
1) Normalized Difference Vegetation Index (NDVI)
2) Enhanced Vegetation Index (EVI)
3) Unified Normalized Difference Vegetation Index (kNDVI)
4) Modified Soil-Adjusted Vegetation Index (MSAVI)


Enter a number:  2


In [33]:
# Search image collection
s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .select('B2','B3','B4','B6','B8','B11','B12','QA60','SCL') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage)) \

s2_preprocessed = s2.preprocess()

# Calculate vegetation index, Generate median composite and clip to stufy area
vi_composite = s2_preprocessed.spectralIndices([vi_index]).median().clip(roi)


# Create classified image by summing values that are either less or greater than defined thresholds
vi_classes = vi_composite.select(vi_index).gte(thresholds_vi).reduce('sum').toInt()
        

# Mask out non-vegetated areas
bin_mask = vi_classes.eq(0)
mask = bin_mask.eq(0)
masked = vi_classes.updateMask(mask)
vi_classes = masked

# Display interactively
vigour_map = geemap.Map()
left_layer = geemap.ee_tile_layer(vi_classes, ndvi_class_viz, 
                                  'Health (' + start_date + ' to ' + end_date + ')')
right_layer = geemap.ee_tile_layer(vi_composite, rgb_viz, 
                                  'RGB (' + start_date + ' to ' + end_date + ')')
vigour_map.split_map(left_layer = left_layer, right_layer = right_layer)
vigour_map.centerObject(roi, 12)
vigour_map.add_legend(title='Vegetation Canopy Health', legend_dict=legend_dict, draggable=False, position='bottomleft')
vigour_map

Map(center=[2.9774794713318813, 32.394373134471905], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Vegetation yield

Vegetation yield pertains to the amount of harvested crops or forage generated by a specific land area over a given period, typically quantified by the weight or volume of the harvest.

**Leaf Area Index (LAI):**,serving as a measure of leaf material in a canopy, provides information on leaf area (m²) per 100 m², offering a measure to estimate the yield of vegetated areas. Higher LAI values indicate areas with greater potential yield, and the resulting map can serve as an indicator to identify regions where soil may lack sufficient fertilization. (Source: [Airborne Multi-Spectral Data for Quantifying Leaf Area Index](https://forskning.ruc.dk/en/publications/airborne-multi-spectral-data-for-quantifying-leaf-area-index-nitr))

In [22]:
def cal_lai(image):
    # Calculate EVI
    evi = image.expression('float(2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1)))', {
        'NIR': image.select('B8'),
        'RED': image.select('B4'),
        'BLUE': image.select('B2')
    }).rename('evi')
    
    # Calculate LAI using the EVI
    lai = image.expression('float(3.618 * evi - 0.118)', {
        'evi': evi
    }).rename('LAI')
    
    return image.addBands(evi).addBands(lai)

# Search image collection
s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .select(['B2', 'B3', 'B4', 'B6', 'B8', 'B11', 'B12', 'QA60', 'SCL']) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage)) \
    .preprocess()

s2_index = s2.map(cal_lai)

lai_composite= s2_index.median().clip(roi)
lai_classes = lai_composite.select('LAI').gte(thresholds_lai).reduce('sum').toInt()

# Mask out non-vegetated areas
bin_mask = lai_classes.eq(0)
mask = bin_mask.eq(0)
masked_lai = lai_classes.updateMask(mask)
lai_classes = masked_lai

# Display interactively
lai_map = geemap.Map()
left_layer = geemap.ee_tile_layer(lai_classes, lai_class_viz, 
                                  'Health (' + start_date + ' to ' + end_date + ')')
right_layer = geemap.ee_tile_layer(lai_composite, rgb_viz, 
                                   'RGB (' + start_date + ' to ' + end_date + ')')
lai_map.split_map(left_layer=left_layer, right_layer=right_layer)
lai_map.centerObject(roi, 12)
lai_map.add_legend(title='Vegetation yield', legend_dict=lai_legend_dict, draggable=False, position='bottomleft')
lai_map


Map(center=[2.9774794713318813, 32.394373134471905], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Canopy Nitrogen Content 

The assessment of canopy nitrogen content involves examine the concentration and distribution of nitrogen within a plant's foliage, facilitating informed decision-making in crop management, fertilization practices, and overall plant health.

**Canopy Chlorophyll Content Index (CCCI):** Is an index which is use to monitor the Chlorophyll content of the plants. In cases of nitrogen malnourishment, the plant growth process is disrupted, leading to a halt in chlorophyll development. Monitoring the CCCI aids in detecting nitrogen deficiencies, allowing for timely fertilizer application at precise locations in agricultural fields.


In [23]:
def add_ccci(image):
    ccci = image.expression(
        'float(((NIR - RE) / (NIR + RE))/((NIR - R) / (NIR + R)))', {
            'NIR': image.select('B8'),
            'R': image.select('B4'),
            'RE': image.select('B5')
        }).rename('ccci')
    return image.addBands(ccci)

# Search image collection
s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B8', 'B9', 'B11', 'B12', 'QA60', 'SCL']) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage)) \
    .preprocess()

# Apply CCCI calculation to each image in the collection
s2_cci = s2.map(add_ccci)

# Calculate the median composite
composite = s2_cci.median().clip(roi)

# Create classified image by summing values that are either less or greater than defined thresholds
ccci_classes = composite.select('ccci').gte(thresholds_ccci).reduce('sum').toInt()

# Mask out non-vegetated areas
bin_mask = ccci_classes.eq(0)
mask = bin_mask.eq(0)
masked_ccci = ccci_classes.updateMask(mask)
ccci_classes = masked_ccci

# Display interactively
ccci_map = geemap.Map()
ccci_layer = geemap.ee_tile_layer(ccci_classes, ccci_class_viz, 
                                  'CCCI (' + start_date + ' to ' + end_date + ')')
rgb_layer = geemap.ee_tile_layer(composite, rgb_viz, 
                                 'RGB (' + start_date + ' to ' + end_date + ')')
ccci_map.split_map(left_layer=ccci_layer, right_layer=rgb_layer)
ccci_map.centerObject(roi, 12)
ccci_map.add_legend(title='Canopy Chlorophy ll Content', legend_dict=ccci_legend_dict, draggable=False, position='bottomleft')
ccci_map


Map(center=[2.9774794713318813, 32.394373134471905], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Canopy moisture/ water content

**Normalized Difference Moisture Index (NDMI):** NDMI is a vegetation index that is particularly sensitive to variations in vegetation water content. It enables prompt identification of areas within a farm or field that may be undergoing water stress.

In [24]:
def cal_evi(image):
    evi = image.expression('float(2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1)))', {
        'NIR': image.select('B8'),
        'RED': image.select('B4'),
        'BLUE': image.select('B2')
    }).rename('evi')
    
    # Calculate LAI using the EVI
    lai = image.expression('float(3.618 * evi - 0.118)', {
        'evi': evi
    }).rename('lai')
    
    return image.addBands(evi).addBands(lai)

#seaarch for images

s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .select('B2','B3','B4','B6','B8','B11','B12','QA60','SCL') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage)) \
    .preprocess()

s2_index = s2.map(cal_evi)
nmdi_composite = s2_index.spectralIndices(['NDMI']).median().clip(roi)


# Create classified image by summing values that are either less or greater than defined thresholds
nmdi_classes = nmdi_composite.select('NDMI').gte(thresholds_nmdi_veg).reduce('sum').toInt()
        

# Mask out non-vegetated areas
bin_mask = nmdi_composite.select('lai').gte(1)
mask = bin_mask.eq(1)
masked = nmdi_classes.updateMask(mask)
nmdi_classes = masked


# Display interactively
moisture_map = geemap.Map()
left_layer = geemap.ee_tile_layer(nmdi_classes, nmdi_class_viz, 
                                  'Water (' + start_date + ' to ' + end_date + ')')
right_layer = geemap.ee_tile_layer(nmdi_composite, rgb_viz, 
                                  'RGB (' + start_date + ' to ' + end_date + ')')
moisture_map.split_map(left_layer = left_layer, right_layer = right_layer)
moisture_map.centerObject(roi, 12)
moisture_map.add_legend(title='Vegetation Canopy moisture', legend_dict=nmdi_legend_dict, draggable=False, position='bottomleft')
moisture_map

Map(center=[2.9774794713318813, 32.394373134471905], controls=(ZoomControl(options=['position', 'zoom_in_text'…

### Canopy fire Potential

Canopy fire potential index estimates the susceptibility of vegetation canopies, such as forests or grasslands, to catch fire under specific conditions. This index is a linear combination of relative greeness (RG) and the NMDI.

References:
    Caccamo, G., Chisholm, L.A., Bradstock, R.A., Puotinen, M.L. and Pippen, B.G., 2011. Monitoring live fuel moisture content of heathland, shrubland and sclerophyll forest in south-eastern Australia using MODIS data. International Journal of Wildland Fire, 21(3), pp.257-269.


In [25]:
# Define constants
vi = 'evi'
x_years= 5
end_date_datetime = datetime.strptime(end_date, '%Y-%m-%d')
current_year = end_date_datetime.year
current_month = end_date_datetime.month


# Function to calculate EVI and LAI
def add_indices(image):
    evi = image.expression('float(2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1)))', {
        'NIR': image.select('B8'),
        'RED': image.select('B4'),
        'BLUE': image.select('B2')
    }).rename('evi')

    # Calculate LAI using the EVI
    lai = image.expression('float(3.618 * evi - 0.118)', {
        'evi': evi
    }).rename('lai')
    
    # Calculate NMDI for soil 
    nmdi_soil = image.expression('float(0.9 - ((NIR - (SWIR1 - SWIR2))/(NIR + (SWIR1 - SWIR2))))', {
            'NIR': image.select('B8'),
            'SWIR1': image.select('B11'),
            'SWIR2': image.select('B12'),
            }).rename('nmdi_soil')
    
    # Calculate NMDI for vegetation
    nmdi_veg = image.expression('float((NIR - (SWIR1 - SWIR2))/(NIR + (SWIR1 - SWIR2)))', {
            'NIR': image.select('B8'),
            'SWIR1': image.select('B11'),
            'SWIR2': image.select('B12'),
            }).rename('nmdi_veg')

    return image.addBands(evi).addBands(lai).addBands(nmdi_soil).addBands(nmdi_veg)




# Image collection for current VI
start = ee.Date.fromYMD(current_year, current_month, 1)
end = ee.Date.fromYMD(current_year, current_month, calendar.monthrange(current_year, current_month)[1])

s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(roi) \
    .filterDate(start, end) \
    .select('B2','B3','B4','B8','B11','B12','QA60','SCL') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage)) \
    .preprocess()

# Create median composite of current VIs
composite = s2.map(add_indices).median().clip(roi)
s2_vi = composite.select(vi)
mask = composite.select('lai').lt(1)
s2_ndmi_soil = composite.select('nmdi_soil').updateMask(mask)
mask = composite.select('lai').gte(1)
s2_ndmi_veg = composite.select('nmdi_veg').updateMask(mask)



# Image collection for minimum SI
s2_vi_min_max = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(roi) \
    .filterDate(ee.Date.fromYMD(current_year - x_years, current_month, 1), start) \
    .select('B2','B3','B4','B8','B11','B12','QA60','SCL') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage)) \
    .preprocess()

# Create median composite of minimum and maximum VI
composite_min = s2_vi_min_max.map(add_indices).min().clip(roi)
composite_max = s2_vi_min_max.map(add_indices).max().clip(roi)
s2_vi_min = composite_min.select(vi)
s2_vi_max = composite_max.select(vi)

# Calculate the relative greenness
rg = (s2_vi.subtract(s2_vi_min)).divide(s2_vi_max.subtract(s2_vi_min)) \
    .rename('rel_green')

# Calculate the fire potential index
fpi_soil = (rg.subtract(s2_ndmi_soil)).divide(s2_ndmi_soil)
fpi_veg = (rg.subtract(s2_ndmi_veg)).divide(s2_ndmi_veg)

# Create classified image by summing values that are either less or greater than defined thresholds
fpi_soil_classes = fpi_soil.gt(thresholds_nmdi_soil).reduce('sum').toInt()
fpi_veg_classes = fpi_veg.gt(thresholds_nmdi_veg).reduce('sum').toInt()
fpi = ee.ImageCollection([fpi_soil_classes, fpi_veg_classes]).mosaic()

# Mask out non-vegetated areas (i.e., where NDVI/EVI < 0.2)
bin_mask = s2_vi.lt(0.2).reduce('sum').toInt()
mask = bin_mask.eq(0)
masked = fpi.updateMask(mask)
fpi = masked

# Display interactively
fire_map = geemap.Map()
left_layer = geemap.ee_tile_layer(fpi, fpi_class_viz, 'Fire Potential Index')
right_layer = geemap.ee_tile_layer(composite, rgb_viz, 'RGB')
fire_map.split_map(left_layer=left_layer, right_layer=right_layer)
fire_map.centerObject(roi, 12)
fire_map.add_legend(title='Vegetation fire potential', legend_dict=fpi_legend_dict, draggable=False, position='bottomleft')

fire_map


Map(center=[2.9774794713318813, 32.394373134471905], controls=(ZoomControl(options=['position', 'zoom_in_text'…